## Data Visualization: Effectiveness, Expressiveness and an Alternate Ecnoding 

Visual 3:

!["viz3"](assets/how_rate_resized.png)

Let's try and recreate the viz:

In [6]:
import pandas as pd
import altair as alt
import numpy as np
import math

In [7]:
sw = pd.read_csv('assets/StarWars.csv', encoding='latin1')

In [8]:
sw = sw.rename(columns={'Have you seen any of the 6 films in the Star Wars franchise?':'seen_any_movie',
                        'Do you consider yourself to be a fan of the Star Wars film franchise?': 'fan',
                        'Which of the following Star Wars films have you seen? Please select all that apply.' : 'seen_EI',
                        'Unnamed: 4' : 'seen_EII',
                        'Unnamed: 5' : 'seen_EIII',
                        'Unnamed: 6' : 'seen_EIV',
                        'Unnamed: 7' : 'seen_EV',
                        'Unnamed: 8' : 'seen_EVI',
                        'Please rank the Star Wars films in order of preference with 1 being your favorite film in the franchise and 6 being your least favorite film.' : 'rank_EI',
                        'Unnamed: 10' : 'rank_EII',
                        'Unnamed: 11' : 'rank_EIII',
                        'Unnamed: 12' : 'rank_EIV',
                        'Unnamed: 13' : 'rank_EV',
                        'Unnamed: 14' : 'rank_EVI',
                        'Please state whether you view the following characters favorably, unfavorably, or are unfamiliar with him/her.' : 'Han Solo',
                        'Unnamed: 16' : 'Luke Skywalker',
                        'Unnamed: 17' : 'Princess Leia Organa',
                        'Unnamed: 18' : 'Anakin Skywalker',
                        'Unnamed: 19' : 'Obi Wan Kenobi',
                        'Unnamed: 20' : 'Emperor Palpatine',
                        'Unnamed: 21' : 'Darth Vader',
                        'Unnamed: 22' : 'Lando Calrissian',
                        'Unnamed: 23' : 'Boba Fett',
                        'Unnamed: 24' : 'C-3P0',
                        'Unnamed: 25' : 'R2 D2',
                        'Unnamed: 26' : 'Jar Jar Binks',
                        'Unnamed: 27' : 'Padme Amidala',
                        'Unnamed: 28' : 'Yoda',
                       })
sw = sw.drop([0])

In [9]:
episodes = ['EI', 'EII', 'EIII', 'EIV', 'EV', 'EVI']
names = {
    'EI' : 'The Phantom Meanance', 'EII' : 'Attack of the clones', 'EIII' : 'Revenge of the Sith', 
    'EIV': 'A New Hope', 'EV': 'The Empire Strikes Back', 'EVI' : 'The Return of the Jedi'
}

names_l = [names[ep] for ep in episodes]

print("sort order: ",names_l)

sort order:  ['The Phantom Meanance', 'Attack of the clones', 'Revenge of the Sith', 'A New Hope', 'The Empire Strikes Back', 'The Return of the Jedi']


In [10]:
seen_at_least_one = sw.dropna(subset=['seen_' + ep for ep in episodes],how='all')
total = len(seen_at_least_one)

In [11]:
# Recreate this image using altair here (10 POINTS)

alt.themes.enable('fivethirtyeight')
cleanup1=sw[['RespondentID','seen_EI', 'seen_EII',
       'seen_EIII', 'seen_EIV', 'seen_EV', 'seen_EVI', 'rank_EI', 'rank_EII',
       'rank_EIII', 'rank_EIV', 'rank_EV', 'rank_EVI']]
cleanup1.rename(columns={'rank_EI':'The Phantom Meanance', 'rank_EII':'Attack of the clones', 
                         'rank_EIII':'Revenge of the Sith','rank_EIV':'A New Hope', 
                         'rank_EV':'The Empire Strikes Back', 'rank_EVI':'The Return of the Jedi'},inplace=True)
cleanup1.dropna(inplace=True)
cleanup1=cleanup1.melt(id_vars=['RespondentID','seen_EI', 'seen_EII',
       'seen_EIII', 'seen_EIV', 'seen_EV', 'seen_EVI'])
cleanup1.value=cleanup1.value.astype(int)
cleanup1.dtypes


cleanup1.value=cleanup1.value.astype(int)

cleanup_top=cleanup1[cleanup1['value'] <3]
cleanup_top=cleanup_top[['variable','value']]
cleanup_mid=cleanup1[(cleanup1['value']>2) & (cleanup1['value'] <5)]
cleanup_mid=cleanup_mid[['variable','value']]
cleanup_bot=cleanup1[(cleanup1['value']>4)]
cleanup_bot=cleanup_bot[['variable','value']]

final_top=[]
final_mid=[]
final_bot=[]
for mname in names_l:
    final_top.append(round((len(cleanup_top[cleanup_top.variable==mname]))/(len(cleanup1[cleanup1.variable==mname])),2))
    final_mid.append(round((len(cleanup_mid[cleanup_mid.variable==mname]))/len(cleanup1[cleanup1.variable==mname]),2))
    final_bot.append(round((len(cleanup_bot[cleanup_bot.variable==mname]))/len(cleanup1[cleanup1.variable==mname]),2))

tuples_top = list(zip(names_l,final_top))
tuples_mid = list(zip(names_l,final_mid))
tuples_bot = list(zip(names_l,final_bot))

df_top = pd.DataFrame(tuples_top, columns = ['Name', 'Percentage'])
df_mid = pd.DataFrame(tuples_mid, columns = ['Name', 'Percentage'])
df_bot = pd.DataFrame(tuples_bot, columns = ['Name', 'Percentage'])


hi=alt.Chart(df_top).mark_bar(
    opacity=0.8,
    stroke='black',
    strokeWidth=0,
    color='green'
).encode(
    y = alt.Y('Name:N',axis=alt.Axis(ticks=True,gridOpacity=0),title=None,sort=names_l),
    x = alt.X('Percentage:Q',axis=alt.Axis(labels=False,ticks=False,gridOpacity=0,domain=False),title=None)
)
hello = alt.Chart(df_top).mark_text(dx=3, dy=0, color='green',align='left',fontSize=13).encode(
    y=alt.Y('Name', stack='zero',sort=names_l),
    x=alt.X('Percentage:Q'),
    text=alt.Text('Percentage:Q',format="1.0%")
)
# raise NotImplementedError()
top = (hi+hello).properties(height=150,width=100,
    # add a title
    title={"text":"Top Third",
           "fontSize":12,
           "anchor":"middle",
           "fontWeight":"bold"
           })

###############################################################################################
hi1=alt.Chart(df_mid).mark_bar(
    opacity=0.8,
    stroke='black',
    strokeWidth=0,
    color='#008fd5'
).encode(
    y = alt.Y('Name:N',axis=alt.Axis(ticks=False,gridOpacity=0,labels=False),title=None,sort=names_l),
    x = alt.X('Percentage:Q',axis=alt.Axis(labels=False,ticks=False,gridOpacity=0),title=None)
)
hello1 = alt.Chart(df_mid).mark_text(dx=3, dy=0, color='#008fd5',align='left',fontSize=13).encode(
    y=alt.Y('Name', stack='zero',sort=names_l,axis=alt.Axis(ticks=False,grid=False,domain=False)),
    x=alt.X('Percentage:Q',axis=alt.Axis(ticks=False,grid=False,domain=False)),
    text=alt.Text('Percentage:Q',format="1.0%")
)
# raise NotImplementedError()
mid = (hi1+hello1).properties(height=150,width=60,
    # add a title
    title={"text":"Middle Third",
           "fontSize":12,
           "anchor":"middle",
           "fontWeight":"bold"
           })
########################################################################
hi2=alt.Chart(df_bot).mark_bar(
    opacity=0.8,
    stroke='black',
    strokeWidth=0,
    color='red'
).encode(
    y = alt.Y('Name:N',axis=alt.Axis(ticks=False,gridOpacity=0,labels=False),title=None,sort=names_l),
    x = alt.X('Percentage:Q',axis=alt.Axis(labels=False,ticks=False,gridOpacity=0),title=None)
)
hello2 = alt.Chart(df_bot).mark_text(dx=3, dy=0, color='red',align='left',fontSize=13).encode(
    y=alt.Y('Name', stack='zero',sort=names_l,axis=alt.Axis(ticks=False,grid=False,domain=False)),
    x=alt.X('Percentage:Q',axis=alt.Axis(ticks=False,grid=False,domain=False)),
    text=alt.Text('Percentage:Q',format="1.0%")
)
# raise NotImplementedError()
bot = (hi2+hello2).properties(height=150,width=90,
    # add a title
    title={"text":'Bottom Third',
           "fontSize":12,
           "anchor":"middle",
           "fontWeight":"bold"
           })
####top###########################################################################################
(top|mid|bot).configure_axis(
    labelFontSize=11,
    titleFontSize=20,
#     labelFontWeight='bold',
    grid=False).configure_view(
    strokeWidth=0).properties(
    title={"text":"How People Rate the 'Star Wars' Movies?",
           
           "subtitle":["How often each film was rated in the top, middle, and bottom order ","(by 471 respondents who have seen all six films)"],
           "subtitleColor": "black",
           "subtitleFontSize":18
          }).configure_scale(
    bandPaddingInner=0.2
)


alt.HConcatChart(...)

Now, let's try an alternate visualisation for this chart, and then look through the visual choices made in both the cases

In [13]:
df = pd.DataFrame([['The Phantom Menace',0.16,0.37,0.46],['Attack of the Clones',.14,.29,.57],['Revenge of the Sith',.13,.40,.47],['A New Hope',.50,.31,.19],['The Empire Strikes Back',.64,.22,.14],['Return of the Jedi',.43,.14,.17]],columns=['Movie Names','tt','mt','bt'])

In [14]:
df['a']=0

In [15]:
df = df.melt(id_vars='Movie Names')
df

,Movie Names,variable,value
0,The Phantom Menace,tt,0.16
1,Attack of the Clones,tt,0.14
2,Revenge of the Sith,tt,0.13
3,A New Hope,tt,0.50
4,The Empire Strikes Back,tt,0.64
5,Return of the Jedi,tt,0.43
6,The Phantom Menace,mt,0.37
7,Attack of the Clones,mt,0.29
8,Revenge of the Sith,mt,0.40
9,A New Hope,mt,0.31


In [16]:
hi=alt.Chart(df).transform_filter(
    alt.datum.variable == 'tt'
).mark_circle(
    opacity=0.8,
    stroke='black',
    strokeWidth=1,
).encode(
    y = alt.Y('Movie Names:N',title=None),
    x = alt.X('value:Q',axis=alt.Axis(labels=False,ticks=False),title=None),
    size = alt.Size('value:Q',
        scale=alt.Scale(range=[0, 5000,1000])
                    ,legend=None
    ),
    color = alt.Color('Movie Names:N',legend=None)
).properties(height=150,width=300,title='TOP THIRD'
)
hello = alt.Chart(df).transform_filter(
    alt.datum.variable == 'tt'
).mark_text(dx=1, dy=0, color='white',align='center',fontSize=13).encode(
    y=alt.Y('Movie Names', stack='zero',sort='-x'),
    x=alt.X('value:Q'),
    text=alt.Text('value:Q',format="1.0%"),
    

).properties(height=150,width=300)
##
hi1=alt.Chart(df).transform_filter(
    alt.datum.variable == 'mt'
).mark_circle(
    opacity=0.8,
    stroke='black',
    strokeWidth=1,
).encode(
    y = alt.Y('Movie Names:N',title=None),
    x = alt.X('value:Q',axis=alt.Axis(labels=False,ticks=False),title=None),
    size = alt.Size('value:Q',
        scale=alt.Scale(range=[0, 5000,1000])
                    ,legend=None
    ),
    color = alt.Color('Movie Names:N')
).properties(height=150,width=300,title='MIDDLE THIRD'
)
hello1 = alt.Chart(df).transform_filter(
    alt.datum.variable == 'mt'
).mark_text(dx=1, dy=0, color='white',align='center',fontSize=13).encode(
    y=alt.Y('Movie Names', stack='zero',sort='-x'),
    x=alt.X('value:Q'),
    text=alt.Text('value:Q',format="1.0%"),
    

).properties(height=150,width=300)
##
##
hi2=alt.Chart(df).transform_filter(
    alt.datum.variable == 'bt'
).mark_circle(
    opacity=0.8,
    stroke='black',
    strokeWidth=1,
).encode(
    y = alt.Y('Movie Names:N',title=None),
    x = alt.X('value:Q',axis=alt.Axis(labels=False,ticks=False),title=None),
    size = alt.Size('value:Q',
        scale=alt.Scale(range=[0, 5000,1000])
                    ,legend=None
    ),
    color = alt.Color('Movie Names:N',legend=None)
).properties(height=150,width=300,title='BOTTOM THIRD'
)
hello2 = alt.Chart(df).transform_filter(
    alt.datum.variable == 'bt'
).mark_text(dx=1, dy=0, color='white',align='center',fontSize=13).encode(
    y=alt.Y('Movie Names', stack='zero',sort='-x'),
    x=alt.X('value:Q'),
    text=alt.Text('value:Q',format="1.0%"),
    

).properties(height=150,width=300)
##
hi+hello&hi1+hello1&hi2+hello2
# hi 

alt.VConcatChart(...)

![1.3](assets/1pt3_1.png)
1. In the created viz the color coding is based on movie names while in the case of the fivethirtyeight the colors are coded by the category they belong to.
2. With a fewer color to store in the mind the 538viz gets a decent edge over the created viz.
3. It's much easier to perceive the 538 viz and Steven's law comes into play here given that lengths of bars are easier to perceive as compared to areas.
4. Text encoding of percentage is even more essential in the case of the created viz as it would help in separating the ambiguity with regard to any possible circle overlap.
5. One component of the Gestalt psychology, simplest and most stable representations are favoured indicates to the 538 viz being the better one.
6. The 538 viz would do much better on preattentive processing.

- In all the fivethirtyeight viz has the right amount of inforamtion, has a good placement and consolidation for the eye to view, and has a color coding that reinforces the follow-thorugh of the different categories.